In [1]:
#Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import hvplot.pandas
import requests
import json
from pprint import pprint

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

#Upload from csv
data_set_path = "school_attendance.csv"
data_set_2_path = "all_students_df.csv"
zip_codes_path = "zip_codes.csv"
#population_set_path = "school_attendance.csv"

#Read data and results
data_set_2_df = pd.read_csv(data_set_2_path)
zip_codes_df = pd.read_csv(zip_codes_path, converters={"Zip Code":str})
data_set_df = pd.read_csv(data_set_path)

#population_df = pd.read_csv(population_set_path)
data_set_df.head(10)

,District code,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate,Reporting period,Date update
0,00000CT,Connecticut,NaN,All Students,500285,0.9169,496092.0,0.9294,508346.0,0.9479,June 2022,07/22/2022
1,00000CT,Connecticut,Homelessness,Students Experiencing Homelessness,1814,0.8348,1735.0,0.8155,3916.0,0.8884,June 2022,07/22/2022
2,00000CT,Connecticut,Students With Disabilities,Students With Disabilities,78417,0.8899,76487.0,0.8946,80365.0,0.9277,June 2022,07/22/2022
3,00000CT,Connecticut,Free/Reduced Lunch,Free Meal Eligible,168984,0.8851,176225.0,0.8861,193706.0,0.9314,June 2022,07/22/2022
4,00000CT,Connecticut,Free/Reduced Lunch,Reduced Price Meal Eligible,29905,0.9184,30886.0,0.9299,27507.0,0.9518,June 2022,07/22/2022
5,00000CT,Connecticut,Free/Reduced Lunch,Free/Reduced Price Meal Eligible,198889,0.8901,207111.0,0.8927,221213.0,0.9340,June 2022,07/22/2022
6,00000CT,Connecticut,English Learners,English Learners,43571,0.8976,40619.0,0.8948,45413.0,0.9389,June 2022,07/22/2022
7,00000CT,Connecticut,Race/Ethnicity,All other races,48700,0.9314,47339.0,0.9483,47260.0,0.9559,June 2022,07/22/2022
8,00000CT,Connecticut,Race/Ethnicity,Black or African American,63099,0.8941,62267.0,0.8931,64200.0,0.9401,June 2022,07/22/2022
9,00000CT,Connecticut,Race/Ethnicity,Hispanic/Latino of any race,146298,0.8935,138260.0,0.8975,136953.0,0.9362,June 2022,07/22/2022


In [2]:
#Attempt to clean out populations
all_students_df = data_set_df.loc[(data_set_df["Student group"]=="All Students")]
all_students_df = all_students_df[["District name", "2021-2022 student count - year to date"]]
all_students_df["District name"] = all_students_df["District name"].str.upper()
all_students_df

,District name,2021-2022 student count - year to date
0,CONNECTICUT,500285
13,ANDOVER SCHOOL DISTRICT,161
19,ANSONIA SCHOOL DISTRICT,2139
31,ASHFORD SCHOOL DISTRICT,343
40,AVON SCHOOL DISTRICT,3057
...,...,...
1972,UNIFIED SCHOOL DISTRICT #2,78
1977,CONNECTICUT TECHNICAL EDUCATION AND CAREER SYSTEM,11138
1990,NORWICH FREE ACADEMY DISTRICT,2068
2002,THE GILBERT SCHOOL DISTRICT,421


In [3]:
#Rename columns
zip_codes_df = zip_codes_df.rename(columns={"District Name": "District name"})
zip_codes_df = zip_codes_df[["District name", "Zip Code"]]
zip_codes_df.head()

,District name,Zip Code
0,ACHIEVEMENT FIRST HARTFORD AC,06112
1,AMISTAD ACADEMY DISTRICT,06513
2,ANDOVER SCHOOL DISTRICT,06232
3,ANSONIA SCHOOL DISTRICT,06401
4,AREA COOPERATIVE EDUCATIONAL,06473


In [4]:
#Merge Files
merged_df = pd.merge(all_students_df, zip_codes_df, on="District name", how = "left")
#merged_3_df = merged_df[["District name", "Zip Code"]]
merged_df = merged_df.dropna()
merged_df

,District name,2021-2022 student count - year to date,Zip Code
1,ANDOVER SCHOOL DISTRICT,161,06232
2,ANSONIA SCHOOL DISTRICT,2139,06401
3,ASHFORD SCHOOL DISTRICT,343,06278
4,AVON SCHOOL DISTRICT,3057,06001
5,BARKHAMSTED SCHOOL DISTRICT,199,06063
...,...,...,...
179,THE BRIDGE ACADEMY DISTRICT,268,06608
181,EXPLORATIONS DISTRICT,87,06098
182,AMISTAD ACADEMY DISTRICT,1093,06513
195,UNIFIED SCHOOL DISTRICT #1,140,06109


In [5]:
# Import API key
from config3 import geoapify_key

In [6]:
#Set up base API URL
zip_code = "06473"
country = "United States"
url = f"https://api.geoapify.com/v1/geocode/search?text={zip_code}&format=json&apiKey={geoapify_key}"
geo_data = requests.get(url).json()

pprint(geo_data)


{'query': {'parsed': {'expected_type': 'unknown', 'postcode': '06473'},
           'text': '06473'},
 'results': [{'address_line1': 'North Haven',
              'address_line2': 'North Haven, CT 06473, United States of '
                               'America',
              'bbox': {'lat1': 41.221443578114,
                       'lat2': 41.541443578114,
                       'lon1': -73.019669765131,
                       'lon2': -72.699669765131},
              'city': 'North Haven',
              'country': 'United States',
              'country_code': 'us',
              'county': 'South Central Connecticut Planning Region',
              'datasource': {'attribution': '© OpenStreetMap contributors',
                             'license': 'Open Database License',
                             'sourcename': 'openstreetmap',
                             'url': 'https://www.openstreetmap.org/copyright'},
              'formatted': 'North Haven, CT 06473, United States of America',

In [7]:
zip_code = []
districts = []
for item in merged_df["Zip Code"]:
    zip_code.append(item)
for district in merged_df["2021-2022 student count - year to date"]:
    districts.append(district)
districts

[161,
 2139,
 343,
 3057,
 199,
 2613,
 391,
 3102,
 1734,
 745,
 159,
 2527,
 18482,
 7439,
 2545,
 804,
 77,
 429,
 1462,
 143,
 4119,
 214,
 1493,
 2141,
 75,
 413,
 105,
 1549,
 1873,
 11783,
 4675,
 206,
 1222,
 126,
 839,
 951,
 1713,
 6126,
 2660,
 2597,
 875,
 971,
 2548,
 4660,
 271,
 9229,
 4073,
 157,
 5631,
 1726,
 8471,
 1639,
 4560,
 3108,
 5392,
 63,
 16371,
 129,
 635,
 195,
 2245,
 886,
 2424,
 381,
 808,
 2414,
 6039,
 948,
 445,
 8476,
 4346,
 5204,
 3319,
 1970,
 4208,
 9299,
 4113,
 2096,
 425,
 18028,
 3850,
 3055,
 3601,
 4001,
 68,
 218,
 3130,
 12356,
 3176,
 1014,
 1242,
 1693,
 1860,
 2171,
 1245,
 352,
 1209,
 383,
 1063,
 789,
 4533,
 2483,
 385,
 282,
 86,
 2077,
 99,
 4404,
 247,
 4068,
 1319,
 6171,
 4714,
 248,
 1312,
 15986,
 314,
 1805,
 6724,
 2016,
 795,
 892,
 2200,
 3813,
 6703,
 44,
 3019,
 213,
 5133,
 17806,
 2287,
 2521,
 590,
 9018,
 5863,
 2204,
 5269,
 3578,
 376,
 3734,
 532,
 2960,
 3219,
 1428,
 2121,
 834,
 781,
 335,
 814,
 2132,
 830,

In [8]:
district_data = []
country_code = "us"

i=0
for code in zip_code:
    try:
        #zip_code_url = f"https://api.geoapify.com/v1/geocode/search?text={code}&format=json&apiKey={geoapify_key}"
        zip_code_url = f"https://api.geoapify.com/v1/geocode/search?text={code}%20{country_code}&format=json&apiKey={geoapify_key}"
        zip_code_requests = requests.get(zip_code_url).json()
        print(f"Zip Code URL: {zip_code_url}")
    #Find Lat and Long
        zip_lat = zip_code_requests['results'][0]['lat']
        zip_lng = zip_code_requests['results'][0]['lon']
        zip_code_1 = zip_code_requests["results"][0]["postcode"]
        
       
     # Append the City information into city_data list
        district_data.append({"Zip Code": zip_code_1, 
                          "Lat": zip_lat, 
                          "Lng": zip_lng, 
                            "Population": districts[i]/75 
                          })
    except:
        print(f"not working")
    i=i+1        

Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06232%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06401%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06278%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06001%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06063%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06037%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06524%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06801%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06040%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06268%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06447%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06450%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06457%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06460%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06468%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06370%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06498%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06107%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06516%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06883%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06880%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06109%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06279%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06897%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


In [9]:
district_data_df = pd.DataFrame(district_data)
district_data_df

,Zip Code,Lat,Lng,Population
0,06232,41.732991,-72.375790,2.146667
1,06401,41.342445,-73.072524,28.520000
2,06278,41.890680,-72.171084,4.573333
3,06001,41.791284,-72.866740,40.760000
4,06063,41.918415,-72.977078,2.653333
...,...,...,...,...
165,06608,41.189517,-73.180982,3.573333
166,06098,41.927121,-73.078201,1.160000
167,06513,41.316445,-72.875431,14.573333
168,06109,41.700966,-72.676233,1.866667


In [10]:
map_plot_1 = district_data_df.hvplot.points(
    "Lng",
    "Lat",
    s="Population",
    geo = True,
    tiles = "OSM",
    
)
hvplot.save(map_plot_1, "population.png")
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Population)

# By Attendance rate

In [11]:
#Read data and results
attendance_df = pd.read_csv(data_set_2_path)
zip_codes_df = pd.read_csv(zip_codes_path, converters={"Zip Code":str})
data_set_df = pd.read_csv(data_set_path)

#population_df = pd.read_csv(population_set_path)
data_set_df.head()

,District code,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate,Reporting period,Date update
0,00000CT,Connecticut,NaN,All Students,500285,0.9169,496092.0,0.9294,508346.0,0.9479,June 2022,07/22/2022
1,00000CT,Connecticut,Homelessness,Students Experiencing Homelessness,1814,0.8348,1735.0,0.8155,3916.0,0.8884,June 2022,07/22/2022
2,00000CT,Connecticut,Students With Disabilities,Students With Disabilities,78417,0.8899,76487.0,0.8946,80365.0,0.9277,June 2022,07/22/2022
3,00000CT,Connecticut,Free/Reduced Lunch,Free Meal Eligible,168984,0.8851,176225.0,0.8861,193706.0,0.9314,June 2022,07/22/2022
4,00000CT,Connecticut,Free/Reduced Lunch,Reduced Price Meal Eligible,29905,0.9184,30886.0,0.9299,27507.0,0.9518,June 2022,07/22/2022


In [12]:
#Attempt to clean out populations
all_students_1_df = data_set_df.loc[(data_set_df["Student group"]=="All Students")]
all_students_1_df = all_students_1_df[["District name", "2021-2022 attendance rate - year to date"]]
all_students_1_df["District name"] = all_students_1_df["District name"].str.upper()
all_students_1_df = all_students_1_df.loc[(all_students_1_df["District name"]!="Connecticut")]
all_students_1_df

,District name,2021-2022 attendance rate - year to date
0,CONNECTICUT,0.9169
13,ANDOVER SCHOOL DISTRICT,0.9386
19,ANSONIA SCHOOL DISTRICT,0.9045
31,ASHFORD SCHOOL DISTRICT,0.9309
40,AVON SCHOOL DISTRICT,0.9457
...,...,...
1972,UNIFIED SCHOOL DISTRICT #2,0.9202
1977,CONNECTICUT TECHNICAL EDUCATION AND CAREER SYSTEM,0.8985
1990,NORWICH FREE ACADEMY DISTRICT,0.8940
2002,THE GILBERT SCHOOL DISTRICT,0.8901


In [13]:
#Merge Files
zip_codes_df = zip_codes_df.rename(columns={"District Name": "District name"})
merged_1_df = pd.merge(all_students_1_df, zip_codes_df, on="District name", how = "right")
#merged_3_df = merged_df[["District name", "Zip Code"]]
merged_1_df = merged_1_df.dropna()
merged_1_df

,District name,2021-2022 attendance rate - year to date,NCES LEA ID,State ID,City,State,Zip Code,Zip +4,"SRSA, Not RLIS eligible",LEA Operational Status,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33


# Race = White

In [14]:
#White populations
white_students_df = data_set_df.loc[(data_set_df["Student group"]=="White")]
white_students_df = white_students_df[["District name", "2021-2022 student count - year to date"]]
white_students_df["District name"] = white_students_df["District name"].str.upper()
white_students_df

,District name,2021-2022 student count - year to date
10,CONNECTICUT,242188
16,ANDOVER SCHOOL DISTRICT,134
28,ANSONIA SCHOOL DISTRICT,569
37,ASHFORD SCHOOL DISTRICT,283
49,AVON SCHOOL DISTRICT,1945
...,...,...
1976,UNIFIED SCHOOL DISTRICT #2,33
1987,CONNECTICUT TECHNICAL EDUCATION AND CAREER SYSTEM,4453
1999,NORWICH FREE ACADEMY DISTRICT,1009
2008,THE GILBERT SCHOOL DISTRICT,277


In [15]:
#Rename columns
zip_codes_df = zip_codes_df.rename(columns={"District Name": "District name"})
zip_codes_df = zip_codes_df[["District name", "Zip Code"]]
zip_codes_df.head()

,District name,Zip Code
0,ACHIEVEMENT FIRST HARTFORD AC,06112
1,AMISTAD ACADEMY DISTRICT,06513
2,ANDOVER SCHOOL DISTRICT,06232
3,ANSONIA SCHOOL DISTRICT,06401
4,AREA COOPERATIVE EDUCATIONAL,06473


In [16]:
#Merge Files
merged_1_df = pd.merge(white_students_df, zip_codes_df, on="District name", how = "left")
merged_1_df = merged_1_df.dropna()
merged_1_df

,District name,2021-2022 student count - year to date,Zip Code
1,ANDOVER SCHOOL DISTRICT,134,06232
2,ANSONIA SCHOOL DISTRICT,569,06401
3,ASHFORD SCHOOL DISTRICT,283,06278
4,AVON SCHOOL DISTRICT,1945,06001
5,BARKHAMSTED SCHOOL DISTRICT,176,06063
...,...,...,...
163,REGIONAL SCHOOL DISTRICT 18,1060,06371
164,REGIONAL SCHOOL DISTRICT 19,862,06268
170,LEARN,389,06371
174,EXPLORATIONS DISTRICT,73,06098


In [17]:
zip_code = []
white_count = []
for item in merged_1_df["Zip Code"]:
    zip_code.append(item)
for district in merged_1_df["2021-2022 student count - year to date"]:
    white_count.append(district)
white_count

[134,
 569,
 283,
 1945,
 176,
 2040,
 307,
 1846,
 64,
 569,
 118,
 1763,
 1753,
 3626,
 1863,
 635,
 386,
 1200,
 121,
 3146,
 191,
 1084,
 1767,
 71,
 352,
 87,
 1343,
 1293,
 2824,
 3869,
 178,
 385,
 106,
 616,
 787,
 1493,
 669,
 1350,
 1911,
 677,
 481,
 1879,
 3022,
 232,
 6773,
 2280,
 120,
 3742,
 1447,
 5112,
 1227,
 2340,
 2490,
 1630,
 56,
 1200,
 109,
 563,
 166,
 1737,
 776,
 1725,
 303,
 706,
 2058,
 1875,
 614,
 382,
 1942,
 1987,
 3524,
 2401,
 1199,
 1883,
 1409,
 3319,
 1591,
 381,
 1945,
 2093,
 410,
 2464,
 3295,
 59,
 176,
 2289,
 2952,
 889,
 789,
 888,
 1431,
 1530,
 1400,
 956,
 328,
 875,
 312,
 812,
 649,
 3589,
 1302,
 321,
 221,
 78,
 1383,
 85,
 2674,
 207,
 2901,
 1181,
 4767,
 2341,
 164,
 1106,
 4377,
 253,
 1521,
 1980,
 1675,
 662,
 783,
 1803,
 1900,
 4245,
 1570,
 190,
 3547,
 2156,
 1687,
 2059,
 425,
 5033,
 1592,
 1711,
 4072,
 2351,
 327,
 2774,
 433,
 652,
 725,
 812,
 1623,
 544,
 691,
 263,
 702,
 1488,
 738,
 841,
 1070,
 675,
 1501,
 187,


In [18]:
white_data = []
country_code = "us"

i=0
for code in zip_code:
    try:
        #zip_code_url = f"https://api.geoapify.com/v1/geocode/search?text={code}&format=json&apiKey={geoapify_key}"
        zip_code_url = f"https://api.geoapify.com/v1/geocode/search?text={code}%20{country_code}&format=json&apiKey={geoapify_key}"
        zip_code_requests = requests.get(zip_code_url).json()
        print(f"Zip Code URL: {zip_code_url}")
    #Find Lat and Long
        zip_lat = zip_code_requests['results'][0]['lat']
        zip_lng = zip_code_requests['results'][0]['lon']
        zip_code_1 = zip_code_requests["results"][0]["postcode"]
        
       
     # Append the City information into city_data list
        white_data.append({"Zip Code": zip_code_1, 
                          "Lat": zip_lat, 
                          "Lng": zip_lng, 
                            "Population": white_count[i]/75 
                          })
    except:
        print(f"not working")
    i=i+1        

Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06232%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06401%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06278%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06001%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06063%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06037%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06524%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06801%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06268%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06447%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06450%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06457%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06460%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06468%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06370%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06770%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06516%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06883%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06880%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06109%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06279%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06897%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06098%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06226%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


In [19]:
white_data_df = pd.DataFrame(white_data)
white_data_df

,Zip Code,Lat,Lng,Population
0,06232,41.732991,-72.375790,1.786667
1,06401,41.342445,-73.072524,7.586667
2,06278,41.890680,-72.171084,3.773333
3,06001,41.791284,-72.866740,25.933333
4,06063,41.918415,-72.977078,2.346667
...,...,...,...,...
159,06371,41.327859,-72.304361,14.133333
160,06268,41.802167,-72.254583,11.493333
161,06371,41.327859,-72.304361,5.186667
162,06098,41.927121,-73.078201,0.973333


In [20]:
map_plot_2 = white_data_df.hvplot.points(
    "Lng",
    "Lat",
    s="Population",
    geo = True,
    tiles = "OSM",
    
)
hvplot.save(map_plot_2, "whitepopulation.png")
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Population)

# Race=Black

In [21]:
#Black populations
black_students_df = data_set_df.loc[(data_set_df["Student group"]=="Black or African American")]
black_students_df = black_students_df[["District name", "2021-2022 student count - year to date"]]
black_students_df["District name"] = black_students_df["District name"].str.upper()
black_students_df = black_students_df.drop(8)
black_students_df

,District name,2021-2022 student count - year to date
26,ANSONIA SCHOOL DISTRICT,434
47,AVON SCHOOL DISTRICT,179
65,BERLIN SCHOOL DISTRICT,64
84,BETHEL SCHOOL DISTRICT,128
96,BLOOMFIELD SCHOOL DISTRICT,1273
...,...,...
1962,CAPITAL PREPARATORY HARBOR SCHOOL DISTRICT,490
1970,UNIFIED SCHOOL DISTRICT #1,71
1975,UNIFIED SCHOOL DISTRICT #2,20
1985,CONNECTICUT TECHNICAL EDUCATION AND CAREER SYSTEM,1308


In [22]:
#Rename columns
zip_codes_df = zip_codes_df.rename(columns={"District Name": "District name"})
zip_codes_df = zip_codes_df[["District name", "Zip Code"]]
zip_codes_df.head()

,District name,Zip Code
0,ACHIEVEMENT FIRST HARTFORD AC,06112
1,AMISTAD ACADEMY DISTRICT,06513
2,ANDOVER SCHOOL DISTRICT,06232
3,ANSONIA SCHOOL DISTRICT,06401
4,AREA COOPERATIVE EDUCATIONAL,06473


In [23]:
#Merge Files
merged_2_df = pd.merge(black_students_df, zip_codes_df, on="District name", how = "left")
merged_2_df = merged_2_df.dropna()
merged_2_df

,District name,2021-2022 student count - year to date,Zip Code
0,ANSONIA SCHOOL DISTRICT,434,06401
1,AVON SCHOOL DISTRICT,179,06001
2,BERLIN SCHOOL DISTRICT,64,06037
3,BETHEL SCHOOL DISTRICT,128,06801
4,BLOOMFIELD SCHOOL DISTRICT,1273,06002
...,...,...,...
103,JUMOKE ACADEMY DISTRICT,558,06112
108,THE BRIDGE ACADEMY DISTRICT,147,06608
110,AMISTAD ACADEMY DISTRICT,656,06513
123,UNIFIED SCHOOL DISTRICT #1,71,06109


In [24]:
zip_code = []
black_count = []
for item in merged_2_df["Zip Code"]:
    zip_code.append(item)
for district in merged_2_df["2021-2022 student count - year to date"]:
    black_count.append(district)
black_count

[434,
 179,
 64,
 128,
 1273,
 36,
 97,
 5550,
 572,
 74,
 89,
 135,
 33,
 126,
 785,
 37,
 238,
 68,
 21,
 1840,
 150,
 39,
 27,
 143,
 103,
 302,
 257,
 199,
 180,
 104,
 187,
 41,
 304,
 37,
 1718,
 4852,
 35,
 110,
 1492,
 32,
 956,
 954,
 254,
 102,
 86,
 466,
 1016,
 54,
 45,
 6203,
 220,
 507,
 116,
 85,
 157,
 1768,
 594,
 31,
 32,
 66,
 118,
 41,
 20,
 33,
 86,
 143,
 332,
 240,
 30,
 147,
 276,
 2212,
 1707,
 79,
 65,
 220,
 416,
 316,
 86,
 3876,
 55,
 84,
 779,
 1411,
 29,
 117,
 151,
 37,
 89,
 1310,
 180,
 77,
 24,
 74,
 89,
 81,
 50,
 98,
 558,
 147,
 656,
 71,
 20]

In [25]:
black_data = []
country_code = "us"

i=0
for code in zip_code:
    try:
        #zip_code_url = f"https://api.geoapify.com/v1/geocode/search?text={code}&format=json&apiKey={geoapify_key}"
        zip_code_url = f"https://api.geoapify.com/v1/geocode/search?text={code}%20{country_code}&format=json&apiKey={geoapify_key}"
        zip_code_requests = requests.get(zip_code_url).json()
        print(f"Zip Code URL: {zip_code_url}")
    #Find Lat and Long
        zip_lat = zip_code_requests['results'][0]['lat']
        zip_lng = zip_code_requests['results'][0]['lon']
        zip_code_1 = zip_code_requests["results"][0]["postcode"]
        
       
     # Append the City information into city_data list
        black_data.append({"Zip Code": zip_code_1, 
                          "Lat": zip_lat, 
                          "Lng": zip_lng, 
                            "Population": white_count[i]/75 
                          })
    except:
        print(f"not working")
    i=i+1        

Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06401%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06001%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06037%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06801%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06002%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06043%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06405%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06604%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06484%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06070%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06071%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06489%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06074%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06901%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06615%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06078%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


In [26]:
black_data_df = pd.DataFrame(black_data)
black_data_df

,Zip Code,Lat,Lng,Population
0,06401,41.342445,-73.072524,1.786667
1,06001,41.791284,-72.866740,7.586667
2,06037,41.619115,-72.768949,3.773333
3,06801,41.381800,-73.399575,25.933333
4,06002,41.833113,-72.727265,2.346667
...,...,...,...,...
97,06112,41.790335,-72.694482,8.653333
98,06608,41.189517,-73.180982,47.853333
99,06513,41.316445,-72.875431,17.360000
100,06109,41.700966,-72.676233,4.280000


In [27]:
map_plot_3 = black_data_df.hvplot.points(
    "Lng",
    "Lat",
    s="Population",
    geo = True,
    tiles = "OSM",
    
)
hvplot.save(map_plot_3, "blackpopulation.png")
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Population)